In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
text = """
There are no stars tonight
But those of memory.
Yet how much room for memory there is
In the loose girdle of soft rain.
There is even room enough
For the letters of my mother’s mother,
Elizabeth,
That have been pressed so long
Into a corner of the roof
That they are brown and soft,
And liable to melt as snow.
Over the greatness of such space
Steps must be gentle.
It is all hung by an invisible white hair.
It trembles as birch limbs webbing the air.
"""

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [4]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [5]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [6]:

predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = tf.keras.utils.to_categorical(label, num_classes=total_words)

In [7]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 8, 100)            6800      
                                                                 
 lstm (LSTM)                 (None, 8, 150)            150600    
                                                                 
 dropout (Dropout)           (None, 8, 150)            0         
                                                                 
 lstm_1 (LSTM)               (None, 150)               180600    
                                                                 
 dense (Dense)               (None, 68)                10268     
                                                                 
Total params: 348268 (1.33 MB)
Trainable params: 348268 (1.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
model.fit(predictors, label, epochs=500, verbose=1)

Epoch 1/500
3/3 [==============================] - 4s 50ms/step - loss: 4.2211 - accuracy: 0.0274
Epoch 2/500
3/3 [==============================] - 0s 54ms/step - loss: 4.2100 - accuracy: 0.0685
Epoch 3/500
3/3 [==============================] - 0s 51ms/step - loss: 4.1969 - accuracy: 0.0685
Epoch 4/500
3/3 [==============================] - 0s 53ms/step - loss: 4.1793 - accuracy: 0.0685
Epoch 5/500
3/3 [==============================] - 0s 50ms/step - loss: 4.1400 - accuracy: 0.0685
Epoch 6/500
3/3 [==============================] - 0s 50ms/step - loss: 4.0520 - accuracy: 0.0685
Epoch 7/500
3/3 [==============================] - 0s 53ms/step - loss: 4.0164 - accuracy: 0.0685
Epoch 8/500
3/3 [==============================] - 0s 55ms/step - loss: 4.0082 - accuracy: 0.0685
Epoch 9/500
3/3 [==============================] - 0s 49ms/step - loss: 3.9559 - accuracy: 0.0822
Epoch 10/500
3/3 [==============================] - 0s 44ms/step - loss: 3.9399 - accuracy: 0.0822
Epoch 11/500
3/3 [=

In [9]:
def generate_text(seed_text, next_words, model, max_sequence_len, temperature=1.0):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

        predicted_probs = model.predict(token_list, verbose=0).flatten()

        predicted_probs = np.log(predicted_probs) / temperature
        predicted_probs = np.exp(predicted_probs) / np.sum(np.exp(predicted_probs))

        predicted = np.random.choice(range(total_words), size=1, p=predicted_probs)[0]

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [10]:
seed_text = "From fairest creatures we desire"
generated_text = generate_text(seed_text, 10, model, max_sequence_len, temperature=0.5)
print(generated_text)

From fairest creatures we desire the are brown such space rain rain hair hair hair
